<div>
<img src="figures/svtLogo.png"/>
</div>  

<center><h1>Mathematical Optimization for Engineers</h1></center>
<center><h2>Lab 7 - Interior point method</h2></center>

Let's consider the following linear optimization problem:

\begin{align*}
		\min_{x_1,x_2} x_1 + x_2 \\
		\mbox{s.t. } x_1 \geq -1 \\
								 x_2 \geq -1 \\
\end{align*}

<u>Task 1</u>: Sketch the feasible region for this problem and contour lines of the objective. Where is the optimal solution?

(done on paper)

<u>Task 2</u>: Set up the Lagrangian function and write down the KKT-conditions.

L(x1, x2, l1, l2) = x1 + x2 - l1(x1 + 1) - l2(x2 + 1)<br>
1-l1 = 0 <br>
1-l2 = 0 <br>

KKT conditions: <br>
l1(x1 + 1) = 0 <br>
l2(x2 + 1) = 0 <br>
x1 + 1 $\ge$ 0 <br>
x2 + 1 $\ge$ 0 <br>
l1, l2 $\ge$ 0 <br>


<u>Task 3</u>: Transform the inequalities of the resulting nonlinear equation system into equalities by introducing slack variables.


Take s1 $\ge$ 0, s2 $\ge$ 0: <br>
x1 + 1 $\ge$ 0 => x1 + 1 - s1 = 0 <br>
x2 + 1 $\ge$ 0 => x2 + 1 - s2 = 0 <br>
l1(x1 + 1) = 0 => l1\*s1 = 0<br>
l2(x2 + 1) = 0 => l2\*s2 = 0<br>

The general idea of interior-point methods is to apply the Newton method (for root finding) to solve (approximated) KKT conditions.

<u>Task 4</u>: Change the complementarity constraints by adding a parameter $\tau$ on the right hand side.

Take $\tau$ > 0:  <br>
l1\*s1 = 0 => l1\*s1 = $\tau$ <br>
l2\*s2 = 0 => l2\*s2 = $\tau$ <br>

<u>Task 5</u>: Solve the nonlinear equation system analytically.

1-l1 = 0 => l1 = 1<br>
1-l2 = 0 => l2 = 1<br>

l1\*s1 = $\tau$ => s1 = $\tau$ / l1 <br>
l2\*s2 = $\tau$ => s2 = $\tau$ / l2 <br>

x1 + 1 - s1 = 0  => x1 = s1 - 1 = $\tau$ / l1 - 1<br>
x2 + 1 - s2 = 0  => x2 = s2 - 1 = $\tau$ / l2 - 1<br>

<u>Task 6</u>: Sketch the central path in the domain of $x_1$ and $x_2$ for different values of $\tau$.

(done on paper)

<u>Task 7</u>: Sketch the feasible set of the complementarity constraints for different values of $\tau$.

(done on paper)

<u>Task 8</u>: Write down the Newton step for solving the approximated KKT-conditions.

$\bx$^(k+1) = $\bx$^k - f($\bx$^k)/f'($\bx$^k) => $\delta$$\bx$^k = - f($\bx$^k)/f'($\bx$^k) => f'($\bx$^k)\*$\delta$$\bx$^k = - f($\bx$^k)<br>

If ||$\delta$$\bx$^k|| < $\epsilon$ for a small $\epsilon$, the algorithms stops and returns x_(k+1) <br>
<br>

$\delta$x_(1,k) = x_(1,k+1) − x_(1,k) <br>
$\delta$x_(2,k) = x_(2,k+1) − x_(2,k) <br>
$\delta$l_(1,k) = l_(1,k+1) − l_(1,k) <br>
$\delta$l_(2,k) = l_(2,k+1) − l_(2,k) <br>
$\delta$s_(1,k) = s_(1,k+1) − s_(1,k) <br>
$\delta$s_(2,k) = s_(2,k+1) − s_(2,k) <br>
<br>

$$\left[\begin{array}{cccccc}
    0 & 0 &     0     &     0     & -1  &  0  \\
    0 & 0 &     0     &     0     &  0  & -1  \\
    1 & 0 &    -1     &     0     &  0  &  0  \\
    0 & 1 &     0     &     0     & -1  &  0  \\
    0 & 0 & \lambda_1 &     0     & s_1 &  0  \\
    0 & 0 &     0     & \lambda_2 &  0  & s_2
\end{array}\right]
\cdot \left[\begin{array}{c}
\delta x_{1,k}\\
\delta x_{2,k}\\
\delta s_{1,k}\\
\delta s_{2,k}\\
\delta \lambda_{1,k}\\
\delta \lambda_{2,k}\\
\end{array}\right] =
- \left[\begin{array}{c}
1-\lambda_1\\
1-\lambda_2\\
x_1 + 1 - s_1\\
x_2 + 1 - s_2\\
\lambda_1 s_1\\
\lambda_2 s_2\\
\end{array}\right]
+ \left[\begin{array}{c}
0\\
0\\
0\\
0\\
\tau\\
\tau\\
\end{array}\right]$$


We will solve this system using Python below.

In [ ]:
import autograd
import autograd.numpy as np

# RHS of KKT system
# approximate KKT means the complementarity slackness condition is tau and not 0
def F(Z, tau):
    x1 = Z[0]
    x2 = Z[1]
    s1 = Z[2]
    s2 = Z[3]
    l1 = Z[4]
    l2 = Z[5]
    F = np.array([1-l1,
         1-l2,
         x1+1-s1,
         x2+1-s2,
         l1*s1 - tau,
         l2*s2 - tau
    ])
    return F

Typically, it is very difficult to find a strictly feasible initial point. Here, we are able to pick a random point because the problem is very small.

In [ ]:
 def solve(tau):

    # initial point for original problem
    Z = np.ones(6)
    
    # initial guess for Newton's method
    deltaZ = np.ones(6)
    
    # convergence tolerance
    tol = 1e-5
    
    # f_prime
    jac = autograd.jacobian(F, 0)
    
    it=0
    # KKT conditions are a set of nonlinear equations we need to solve
    # F(x, slack, lambda) = 0 (for exact KKT)
    # Apply Newton's method to iterate to the solution. 
    # F(Z0) + Jacobian(Z0) * deltaZ = 0
    while np.linalg.norm(deltaZ) > tol:
        it=it+1
        
        f_prime = jac(Z,tau)
        f = F(Z,tau)

        # solve linear equation system
        deltaZ = np.linalg.solve(f_prime, -f) 

        Z = Z + deltaZ
        
    return Z, it

In [ ]:
# loop for different values of tau
tau = [1.0, 0.5, 0.25, 0.125, 1e-2, 1e-3, 1e-5]
print("{:<10} {:<20} {:^10}".format('tau','minimum','iter'))
for itau in tau:
    Zopt, it = solve(itau)
    print ("{:<10} [{:^8.4f}, {:^8.4f}] {:<4} {:<20d}".format(itau,Zopt[0],Zopt[1],' ',it))